# Plot results from RePass

In [2]:
import re
import matplotlib.pyplot as plt
import numpy as np

def parse_repass_file(path):
    metrics = {
        "Entailment": None,
        "Contradiction": None,
        "Obligation": None,
        "FinalComposite": None,
    }

    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            m = re.search(r"Average Entailment Score:\s*([0-9.]+)", line)
            if m:
                metrics["Entailment"] = float(m.group(1))
                continue
            m = re.search(r"Average Contradiction Score:\s*([0-9.]+)", line)
            if m:
                metrics["Contradiction"] = float(m.group(1))
                continue
            m = re.search(r"Average Obligation Coverage Score:\s*([0-9.]+)", line)
            if m:
                metrics["Obligation"] = float(m.group(1))
                continue
            m = re.search(r"Average Final Composite Score:\s*([0-9.]+)", line)
            if m:
                metrics["FinalComposite"] = float(m.group(1))
                continue

    return metrics

baseline_path = "results_Au_Irish.txt"
weaviate_path = "results_my.txt"

baseline = parse_repass_file(baseline_path)
weaviate = parse_repass_file(weaviate_path)

metrics = ["Entailment", "Obligation", "FinalComposite"]
baseline_vals = [baseline[m] for m in metrics]
weaviate_vals = [weaviate[m] for m in metrics]

x = np.arange(len(metrics))
width = 0.35

plt.figure(figsize=(10, 6))
plt.bar(x - width/2, baseline_vals, width, label="Baseline")
plt.bar(x + width/2, weaviate_vals, width, label="Weaviate")
plt.xticks(x, metrics)
plt.ylabel("Score")
plt.xlabel("Metric")
plt.title("RePASs Positive Metrics Comparison")
plt.ylim(0, 0.6)
plt.grid(axis="y", linestyle="--", alpha=0.5)

for i, v in enumerate(baseline_vals):
    plt.text(x[i] - width/2, v + 0.01, f"{v:.3f}", ha="center", fontsize=9)
for i, v in enumerate(weaviate_vals):
    plt.text(x[i] + width/2, v + 0.01, f"{v:.3f}", ha="center", fontsize=9)

plt.legend()
plt.tight_layout()
plt.savefig("repass_positive_metrics.png")
plt.close()

contr_labels = ["Baseline", "Weaviate"]
contr_vals = [baseline["Contradiction"], weaviate["Contradiction"]]
y = np.arange(len(contr_labels))

plt.figure(figsize=(8, 4))
plt.barh(y, contr_vals)
plt.yticks(y, contr_labels)
plt.xlabel("Contradiction Score (lower is better)")
plt.title("RePASs Contradiction Comparison")
plt.xlim(0.3, 0.7)
plt.grid(axis="x", linestyle="--", alpha=0.5)

for i, v in enumerate(contr_vals):
    plt.text(v + 0.01, y[i], f"{v:.3f}", va="center", fontsize=10)

plt.tight_layout()
plt.savefig("repass_contradiction.png")
plt.close()
